# Unit Test Writer

### Welcome to the Unit Test Writer - an AI-powered tool that automatically generates comprehensive unit tests for your code across multiple programming languages. Simply paste your code, select your language, and let AI create thorough test suites in seconds!

In [ ]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

In [ ]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)

In [ ]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", ]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

In [ ]:
# Function to generate unit tests
def generate_unit_tests(code, language, model_name):
    """Generate unit tests for the provided code"""
    
    if not code.strip():
        return "Please provide some code to generate tests for."
    
    # Select the appropriate client
    client = clients.get(model_name, openai)
    
    # Create the prompt based on language
    test_frameworks = {
        "Python": "pytest",
        "JavaScript": "Jest",
        "TypeScript": "Jest",
        "Java": "JUnit",
        "C#": "NUnit",
        "Go": "testing package",
        "Ruby": "RSpec",
        "PHP": "PHPUnit",
        "Rust": "built-in test framework"
    }
    
    framework = test_frameworks.get(language, "appropriate testing framework")
    
    prompt = f"""You are a unit test expert. Generate comprehensive unit tests for the following {language} code using {framework}.

                    Code to test:
                    ```{language.lower()}
                    {code}
                    ```

                    Requirements:
                    - Create thorough unit tests covering normal cases, edge cases, and error cases
                    - Use {framework} syntax and best practices
                    - Include clear test names that describe what is being tested
                    - Add comments explaining complex test scenarios
                    - Ensure tests are independent and can run in any order

                    Provide ONLY the test code, no explanations."""

    try:
        # Call the API
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Error generating tests: {str(e)}"


In [ ]:
# Create Gradio interface
def create_interface():
    with gr.Blocks(title="Unit Test Generator") as demo:
        gr.Markdown("# 🧪 Unit Test Generator")
        gr.Markdown("Paste your code, select the language, and generate comprehensive unit tests!")
        
        with gr.Row():
            with gr.Column():
                code_input = gr.TextArea(
                    label="Your Code",
                    placeholder="Paste your code here...",
                    lines=15
                )
                
                with gr.Row():
                    language_dropdown = gr.Dropdown(
                        choices=["Python", "JavaScript", "TypeScript", "Java", "C#", "Go", "Ruby", "PHP", "Rust"],
                        value="Python",
                        label="Language"
                    )
                    
                    model_dropdown = gr.Dropdown(
                        choices=models,
                        value="gpt-5",
                        label="Model"
                    )
                
                generate_btn = gr.Button("Generate Unit Tests", variant="primary")
            
            with gr.Column():
                output = gr.TextArea(
                    label="Generated Unit Tests",
                    lines=15
                )
        
        # Example
        gr.Markdown("### Example")
        gr.Examples(
            examples=[
                ["""def add(a, b):
    return a + b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b""", "Python", "gpt-5"],
                ["""function isPalindrome(str) {
    const cleaned = str.toLowerCase().replace(/[^a-z0-9]/g, '');
    return cleaned === cleaned.split('').reverse().join('');
}""", "JavaScript", "gpt-5"]
            ],
            inputs=[code_input, language_dropdown, model_dropdown]
        )
        
        # Connect the button
        generate_btn.click(
            fn=generate_unit_tests,
            inputs=[code_input, language_dropdown, model_dropdown],
            outputs=output
        )
    
    return demo

In [ ]:
# Launch the interface
demo = create_interface()
demo.launch(share=False)